In [1]:
import numpy as np
import sympy as sp
from scipy.stats import unitary_group

In [2]:
i = sp.I

In [3]:
def generateComplexVector(N):
    real_parts = np.random.randint(-10, 10, N)
    imaginary_parts = np.random.randint(-10, 10, N)
    complex_vector = real_parts + i * imaginary_parts
    return complex_vector

def generateComplexMatrix(N):
    return sp.Matrix([generateComplexVector(N) for _ in range(N)])

def complexProjection(u,v):
    return ((u.H.dot(v))/(u.H.dot(u)) * u).expand()

def GramSchmidt(M):
    matrix = M
    for j in range(1, M.shape[0]):
        col = matrix.col(j)
        for k in range(0, j):
            col -= complexProjection(matrix.col(k),col)
        matrix[j] = col.normalized()
    matrix[0] = matrix.col(0).normalized()
    return matrix.expand()

In [4]:
# For testing
M = generateComplexMatrix(2)
U = GramSchmidt(M)
display(U, (U.H@U).expand())

Matrix([
[-5/7 + 2*I/7, -46*sqrt(493)/3451 - 88*sqrt(493)*I/3451],
[ 2/7 + 4*I/7, 101*sqrt(493)/3451 - 64*sqrt(493)*I/3451]])

Matrix([
[1, 0],
[0, 1]])

In [5]:
def generateQuantumDynamics(N):
    return GramSchmidt(generateComplexMatrix(N))

In [18]:
def checkIfQuantumDynamics(M):
    return (M@M.H).expand().equals(sp.Matrix(np.eye(M.shape[0]), dtype=np.int8))

In [26]:
M = generateQuantumDynamics(3)
display(M, (M@M.H).expand(), checkIfQuantumDynamics(M))

Matrix([
[5*sqrt(93)/93 - 7*sqrt(93)*I/93,  -5*sqrt(2505606)/2505606 + 77*sqrt(2505606)*I/1252803, -46201*sqrt(13109788606)/13109788606 + 10242*sqrt(13109788606)*I/6554894303],
[    sqrt(93)/93 + sqrt(93)*I/31, 322*sqrt(2505606)/1252803 - 273*sqrt(2505606)*I/835202, -16079*sqrt(13109788606)/6554894303 + 70615*sqrt(13109788606)*I/13109788606],
[                 -sqrt(93)*I/31,  112*sqrt(2505606)/417601 - 162*sqrt(2505606)*I/417601,    8983*sqrt(13109788606)/6554894303 - 32451*sqrt(13109788606)*I/6554894303]])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

True

In [27]:
def calcState(matrix, start, k):
    state = start
    for _ in range(abs(k)):
        state = matrix @ state if k > 0 else matrix.T @ state
    return state

In [38]:
def calculateTransitionProbability(M, i, j, T):
    iDirac = np.zeros((M.shape[0], 1))
    iDirac[i] = 1
    state = calcState(M, iDirac, T)
    state = state.normalized()
    return sp.Abs(state[j])**2

In [42]:
M = sp.Matrix([[1, 1], [1, -1]])*1/sp.sqrt(2)
calculateTransitionProbability(M, 0, 1, -3)

0.500000000000000

In [49]:
M = generateQuantumDynamics(4)
display(M)

Matrix([
[                 -7*sqrt(195)*I/195, -278*sqrt(1780935)/1780935 + 452*sqrt(1780935)*I/1780935,   5442*sqrt(16356700685)/1258207745 + 13862*sqrt(16356700685)*I/16356700685, -3022557*sqrt(40061650495945)/40061650495945 + 1340021*sqrt(40061650495945)*I/40061650495945],
[-2*sqrt(195)/65 - 4*sqrt(195)*I/195,   19*sqrt(1780935)/712374 - 2099*sqrt(1780935)*I/3561870,   26337*sqrt(16356700685)/32713401370 - 5437*sqrt(16356700685)*I/6542680274, -1594708*sqrt(40061650495945)/40061650495945 - 1063313*sqrt(40061650495945)*I/40061650495945],
[                  -2*sqrt(195)*I/65, -391*sqrt(1780935)/1187290 + 119*sqrt(1780935)*I/1187290,  1867*sqrt(16356700685)/6542680274 - 103979*sqrt(16356700685)*I/32713401370,  3107784*sqrt(40061650495945)/40061650495945 - 2807019*sqrt(40061650495945)*I/40061650495945],
[   sqrt(195)/65 + 7*sqrt(195)*I/195,     28*sqrt(1780935)/356187 - 14*sqrt(1780935)*I/1780935, 69507*sqrt(16356700685)/16356700685 - 56969*sqrt(16356700685)*I/16356700685,   -107986*sqrt

In [65]:
d0 = calculateTransitionProbability(M, 0, 0, -1).simplify().evalf(6)
d1 = calculateTransitionProbability(M, 0, 1, -1).simplify().evalf(6)
d2 = calculateTransitionProbability(M, 0, 2, -1).simplify().evalf(6)
d3 = calculateTransitionProbability(M, 0, 3, -1).simplify().evalf(6)
display(d0,d1,d2,d3, sum([d0, d1, d2, d3]))

0.251282

0.158112

0.317738

0.272867

1.00000